In [32]:
!pip install numpy
!pip install scipy

import scipy
import numpy as np
from scipy.constants import mu_0
import scipy.special as sp

What we want to approximate

$$ f_x(y) = \frac{1}{|y - x|}$$

where $x$ is a fixed point and $y$ is a variable point. 

In [33]:
def f(x, y):
    rel = y - x
    return 1/np.linalg.norm(rel)

In [34]:

def spherical_to_cartesian(r, theta, phi):
    x = r*np.sin(theta)*np.cos(phi)
    y = r*np.sin(theta)*np.sin(phi)
    z = r*np.cos(theta)
    return x, y, z

def cartesian_to_spherical(x, y, z):
    r = np.sqrt(x**2 + y**2 + z**2)
    theta = np.arccos(z / r)
    phi = np.sign(y)*np.arccos(x/np.sqrt(x**2 + y**2))
    return r, theta, phi

Following https://en.wikipedia.org/wiki/Spherical_multipole_moments

and expressing y, x in spherical coordinates as $y = (r, \theta, \phi)$ and $x = (r^\prime, \theta^\prime, \phi^\prime)$, with the assumption that $r << r^\prime $, we have



$$f(y) = \sum_{l=0}^\infty \sum_{m=-l}^l \frac{1}{r^\prime}\left(\frac{r}{r^\prime}\right)^l\frac{4\pi}{2l+1} Y^*(l, m, \theta^\prime, \phi^\prime)Y(l,m,\theta, \phi)$$

Here $Y$ is a spherical harmonic function, with a standard implementation in scipy. Convention of scipy of spherical harmonic is given by: https://docs.scipy.org/doc/scipy/reference/generated/scipy.special.sph_harm.html, such that we have to switch both l -> m and theta -> phi.

In [35]:
def Y(l,m,theta, phi):
    return sp.sph_harm_y(l,m,theta, phi)

In [36]:
def spherical_approximation(max_l, y, x):
    r, theta, phi = cartesian_to_spherical(*y)
    rp, phip, thetap = cartesian_to_spherical(*x)

    sum = 0
    for l in range(0, max_l+1):
        for m in range(-l, l+1):
            term = 1/rp*(r/rp)**l * 4*np.pi/(2*l+1)* np.conj(Y(l, m,thetap, phip))*Y(l, m, theta, phi)
            sum += term
    return sum.real

In [37]:
x = (1, 0, 0)
y = (0.1, 0.1, 0.1)
max_l = 80
print(f'Approximation: {spherical_approximation(max_l, y, x)}')
print(f'True value: {f(np.array(x), np.array(y))}')

Approximation: 1.097642599896903
True value: 1.0976425998969035


We can do the same for the vector potential, and verify with the analytical solution.

In [54]:
def VP_analytical(r, theta, phi, I):
    rp, thetap, phip, Ix, Iy, Iz = I
    xp, yp, zp = spherical_to_cartesian(rp, thetap, phip)
    x, y, z = spherical_to_cartesian(r, theta, phi)
    I = [Ix, Iy, Iz]

    difference = np.array([x,y,z]) - np.array([xp, yp, zp])
    norm = np.linalg.norm(difference)
    current = np.array(I)

    return current/norm

def VP_spherical(L, r, theta, phi, I):
    rp, phip, thetap, Ix, Iy, Iz = I
    sum = 0
    for l in range(0, L):
        for m in range(-l, l+1):
            term = 1/rp*(r/rp)**l * 4*np.pi/(2*l+1)* np.conj(Y(l, m,thetap, phip))*Y(l, m, theta, phi)
            sum += term
    return sum.real* np.array([Ix, Iy, Iz])

sph = cartesian_to_spherical(.1, 0., 0.1)

I = *cartesian_to_spherical(1, 1, 0),1,3,1

print(f'Approximation: A =  {VP_spherical(50, *sph, I)}')
print(f'True value: A = {VP_analytical(*sph, I)}')

Approximation: A =  [0.74124932 2.22374795 0.74124932]
True value: A = [0.74124932 2.22374795 0.74124932]


We now want to approximate the magnetic field, given by the curl of the vector potential. We can do this by taking the curl of the spherical harmonic expansion of the vector potential. Curl with respect to the spherical coordinates is given by: $$\nabla \times A = \frac{1}{r\sin\theta}\left(\frac{\partial}{\partial \theta}(\sin\theta A_\phi) - \frac{\partial A_\theta}{\partial \phi}\right)\hat{r} + \frac{1}{r}\left(\frac{1}{\sin\theta}\frac{\partial A_r}{\partial \phi} - \frac{\partial}{\partial r}(\sin\theta A_\phi)\right)\hat{\theta} + \frac{1}{r}\left(\frac{\partial}{\partial r}(\sin\theta A_\theta) - \frac{\partial A_r}{\partial \theta}\right)\hat{\phi}$$

In [52]:
def dYdtheta(l,m,theta,phi):
    a =  m*np.cos(theta)/np.sin(theta)*Y(l,m,theta, phi)+np.sqrt((l-m)*(l+m+1)+0*1j)*np.exp(-1j*phi)*Y(l,m+1,theta, phi)
    return a

def dYdphi(l,m,theta,phi):
    return 1j*m*Y(l,m,theta, phi)


delta = 1e-6
l, m = 3,1
theta, phi = np.pi/3, np.pi/4
f = lambda theta: Y(l, m , theta, phi)
derivative_numerically = (f(theta+delta)-f(theta))/delta
derivative_analytically = dYdtheta(l, m, theta, phi)

print('numerically: ', derivative_numerically,'analytically: ', derivative_analytically)
assert np.isclose(derivative_numerically, derivative_analytically)
derivative_numerically = (Y(l, m, theta, phi+delta)-Y(l, m , theta, phi))/delta
derivative_analytically = dYdphi(l, m, theta, phi)

print('numerically: ', derivative_numerically,'analytically: ', derivative_analytically)
assert np.isclose(derivative_numerically, derivative_analytically)
print('All tests passed')

numerically:  (0.828395536139892+0.828395536139892j) analytically:  (0.8283955115293443+0.8283955115293443j)
numerically:  (0.04947668386101611-0.04947663439364147j) analytically:  (0.04947665912864959-0.0494766591286496j)
All tests passed


First we have to convert the Vector potential A to spherical coordinates, which is done by: $$A_r = A_x\sin\theta\cos\phi + A_y\sin\theta\sin\phi + A_z\cos\theta$$
$$A_\theta = A_x\cos\theta\cos\phi + A_y\cos\theta\sin\phi - A_z\sin\theta$$
$$A_\phi = -A_x\sin\phi + A_y\cos\phi$$